In [1]:
import wandb
import pickle
import time
from src.datahandle import *
import time

In [2]:
#hyper
directory = '../../extras/data/A_training_given/training_data/'
training_folder = 'training_data'
training_labels_file= 'training_norm.csv'
#
collected_directory = '../../extras/data/Z_collected_raw/' 
#
quiz_directory = '../../extras/data/C_testing_given/test_data/'
quiz_training_folder = 'test_data'
#configurations
conf_tracking = 0
seed = 0
data_shape = 'original'
parameter_init_scale = 0.01
split= 0.8
batch_size = 64
n_epochs = 5
lr = 0.0001

In [3]:
#dataloading object
training_object= DataLoader(
        directory,
        training_folder,
        training_labels_file
        )
collected_object = DataLoader(
        collected_directory        
        )
collected_train, collected_test = collected_object.LoadCollectedData_info(
        split=split,
        batch_size=1 #for tf this batch size remains as 1
        )
#train test split
train,test = training_object.LoadModelData_info(
        split = split, 
        batch_size =1)#for tf this batch size remains as 1
train_stack = np.vstack((train,collected_train))
test_stack= np.vstack((test[0],collected_test[0]))
example_batch_x,example_batch_y = training_object.Load_batch(train[0], data_shape=data_shape)


In [16]:
loaded_train_X,loaded_train_Y = training_object.Load_batch(train_stack.reshape(-1,4)[0:100],data_shape=data_shape)

In [17]:
loaded_test_X,loaded_test_Y = training_object.Load_batch(test_stack[0:100],data_shape=data_shape)

In [18]:
loaded_train_X.shape ,loaded_train_Y.shape

((100, 240, 320, 3), (100, 2))

In [19]:
image_shape = loaded_train_X[0].shape
image_shape

(240, 320, 3)

In [33]:
#model initialisation
data_augmentation = keras.Sequential(
  [
        layers.experimental.preprocessing.RandomRotation(0.2),
        layers.experimental.preprocessing.RandomZoom(0.2),
        #layers.experimental.preprocessing.RandomContrast(0.2),
        
  ]
)
model = keras.Sequential(
    [
        keras.Input(shape=image_shape),
        data_augmentation,
        layers.Conv2D(32, 5, strides=2, activation="relu"),
        data_augmentation,
        layers.Conv2D(16, 3, padding='same', activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(32, 3, padding='same', activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(64, 3, padding='same', activation='relu'),
        layers.MaxPooling2D(),
        layers.Dropout(0.2),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(2)
    ]
)

In [34]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['accuracy'])

In [ ]:
model_path = 'pkls/tf_cnn_augmented.pkl'
#model = tf.keras.models.load_model(model_path, custom_objects=None, compile=True, options=None)
model.fit(loaded_train_X,loaded_train_Y, epochs=10) 
#tf.keras.models.save_model(
#    model,
#    model_path,
#    overwrite=False,
#    include_optimizer=True,
#    save_format=None,
#    signatures=None,
#    options=None,
#    save_traces=True
#)

Epoch 1/10
4/4 [==============================] - 9s 2s/step - loss: 1.0858 - accuracy: 0.7200
Epoch 2/10
4/4 [==============================] - 8s 2s/step - loss: 0.2707 - accuracy: 0.3700
Epoch 3/10
4/4 [==============================] - 8s 2s/step - loss: 0.1805 - accuracy: 0.2800
Epoch 4/10
4/4 [==============================] - 8s 2s/step - loss: 0.1559 - accuracy: 0.7200
Epoch 5/10


In [31]:
model.evaluate(loaded_test_X,loaded_test_Y)

4/4 [==============================] - 1s 236ms/step - loss: 0.5825 - accuracy: 0.2900


[0.5824778079986572, 0.28999999165534973]

In [13]:
quiz_object= DataLoader(
    quiz_directory,
    quiz_training_folder,
)
quiz_train = quiz_object.LoadQuizData_info()
X,image_order = quiz_object.Load_batch_quiz(quiz_train,data_shape=data_shape)

In [14]:
X.shape

(1020, 240, 320, 3)

In [15]:
prds = model.predict(X)
final_prd = np.column_stack((image_order,prds))
final_ordered = final_prd[final_prd[:, 0].argsort()]
df = pd.DataFrame(final_ordered, columns = ['image_id','angle','speed'])
df = df.astype({'image_id': 'int32'})
df.to_csv('submission.csv', index=False,)